In [9]:
from clean_colored_lyrics import *
from synced_lyrics import *
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from fuzzysearch import find_near_matches
import regex
import datetime
import pandas as pd
import math
import os
from pathlib import Path

In [22]:
with open('../json/urls.json', 'r') as file:
    data = json.load(file)

song_name = 'HIGHLIGHT'
artist = 'seventeen'

song = next((s for s in data if s["name"] == song_name), None)
print(song)
album = song['album']

raw_color_coded = get_raw_color_coded_html(song_name)
colored_lyrics = get_colored_lyrics(raw_color_coded)

export_testing_html(str(raw_color_coded['main_lyrics_body']))

meta_data = get_song_info(song_name, artist, album)
meta_data['unit'] = raw_color_coded['unit']
synced_lyrics = meta_data['syncedLyrics']
print(meta_data['syncedLyrics'])

{'name': 'HIGHLIGHT', 'url': 'https://colorcodedlyrics.com/2016/12/05/seventeen-highlight/', 'album': 'Going Seventeen - SEVENTEEN'}
url found: https://colorcodedlyrics.com/2016/12/05/seventeen-highlight/
found raw color html
requesting  https://lrclib.net/api/get?artist_name=seventeen&track_name=HIGHLIGHT&albumName=Going_Seventeen_-_SEVENTEEN
../official_times/HIGHLIGHT.srt found
[{'start': '00:13:08', 'end': '00:16:28', 'lyric': '흩어진 조각들이 모일 때', 'delta': '00:03:20'}, {'start': '00:16:28', 'end': '00:18:68', 'lyric': '비로소 난 숨을 쉬어', 'delta': '00:02:40'}, {'start': '00:19:58', 'end': '00:22:82', 'lyric': '두 개의 빛 마주할 때', 'delta': '00:03:24'}, {'start': '00:22:82', 'end': '00:25:18', 'lyric': '서로의 내일을 밝혀', 'delta': '00:02:36'}, {'start': '00:26:80', 'end': '00:32:66', 'lyric': '해지고 어둠이 찾아올 땐 오직 난 너의 온도를 기억해', 'delta': '00:05:86'}, {'start': '00:32:66', 'end': '00:36:36', 'lyric': '별들이 감싸 안고 있는 곳', 'delta': '00:03:70'}, {'start': '00:36:36', 'end': '00:40:18', 'lyric': '그곳으로 가고 있어 I’ll be 

In [27]:
print(raw_color_coded)

{'unit': 'Performance', 'color_key': {'bede0d': 'Jun', 'c04737': 'Hoshi', 'a1dad7': 'The8', 'd962b6': 'Dino'}, 'main_lyrics_body': <p><span style="color: #bede0d">흩어진 조각들이 모일 때</span> <span style="color: #bede0d"> 비로소 난 숨을 쉬어</span> <span style="color: #bede0d"> 두 개의 빛 마주할 때</span> <span style="color: #bede0d"> 서로의 내일을 밝혀</span> </p><p><span style="color: #d962b6">해지고 어둠이 찾아올 땐</span> <span style="color: #d962b6"> 오직 난 너의 온도를 기억해</span> <span style="color: #d962b6"> 별들이 감싸 안고 있는 곳</span> <span style="color: #d962b6"> 그곳으로 가고 있어 I’ll be there</span></p> <p><span style="color: #c04737">널 내 맘에 match 하고 싶어</span> <span style="color: #c04737"> 동틀 때 하늘과 햇살처럼</span> <span style="color: #c04737"> 시간과 함께 우린 닮아가</span> <span style="color: #c04737"> 결국에 하나가 될 우릴 봐</span></p> <p> I’m like a bird<span style="color: #a1dad7">날아갈게</span> <span style="color: #a1dad7"> 네 목소리 들리는 곳 어디든지</span> In the air<span style="color: #a1dad7">어디서든 우리</span> <span style="color: #d962b6">마주하는 이 순간</span></p> <p> Hig

In [23]:
for c in colored_lyrics:
    print(c)
print()

for s in synced_lyrics:
    print(s)
print()
print(len(colored_lyrics), len(synced_lyrics))

{'member': ['Jun'], 'lyric': '흩어진 조각들이 모일 때', 'section': 0, 'line': 0}
{'member': ['Jun'], 'lyric': ' 비로소 난 숨을 쉬어', 'section': 0, 'line': 1}
{'member': ['Jun'], 'lyric': ' 두 개의 빛 마주할 때', 'section': 0, 'line': 2}
{'member': ['Jun'], 'lyric': ' 서로의 내일을 밝혀', 'section': 0, 'line': 3}
{'member': ['Dino'], 'lyric': '해지고 어둠이 찾아올 땐', 'section': 1, 'line': 0}
{'member': ['Dino'], 'lyric': ' 오직 난 너의 온도를 기억해', 'section': 1, 'line': 1}
{'member': ['Dino'], 'lyric': ' 별들이 감싸 안고 있는 곳', 'section': 1, 'line': 2}
{'member': ['Dino'], 'lyric': ' 그곳으로 가고 있어 I’ll be there', 'section': 1, 'line': 3}
{'member': ['Hoshi'], 'lyric': '널 내 맘에 match 하고 싶어', 'section': 2, 'line': 0}
{'member': ['Hoshi'], 'lyric': ' 동틀 때 하늘과 햇살처럼', 'section': 2, 'line': 1}
{'member': ['Hoshi'], 'lyric': ' 시간과 함께 우린 닮아가', 'section': 2, 'line': 2}
{'member': ['Hoshi'], 'lyric': ' 결국에 하나가 될 우릴 봐', 'section': 2, 'line': 3}
{'member': ['adlib'], 'lyric': ' I’m like a bird', 'section': 3, 'line': 0}
{'member': ['The8'], 'lyric': '날아갈게',

In [13]:
def look_ahead_color(c_line_og, s_line_og, debug):
    s_line = s_line_og.copy()
    s_lyric = normalize_lyrics(s_line['lyric']).strip()

    c_line = c_line_og.copy()
    c_line['delta'] = 'UNKNOWN'
    c_line['start'] = 'UNKNOWN'
    c_line['end'] = 'UNKNOWN'
    c_lyric = normalize_lyrics(c_line['lyric']).strip()


    if debug: print('s_line_before', s_line)
    if debug: print('c_line_before',c_line)

    found = False
    match_end = 0

    max_l_dist = 5
    if len(c_lyric) < 5 or len(s_lyric) < 5:
        max_l_dist = 1
    elif len(c_lyric) < 10 or len(s_lyric) < 10:
        max_l_dist = int(min(len(c_lyric), len(s_lyric)) / 3)
    
    if debug: print('max dist', max_l_dist, int(min(len(c_lyric), len(s_lyric))))
    find_match = find_near_matches(c_lyric, s_lyric, max_l_dist=max_l_dist)
    if debug: print('find_match: ', find_match)

    if find_match:
        found = find_match[0].start <= 3 and find_match[0].matched
        match_end = find_match[0].end
    else:
        prefix = os.path.commonprefix([c_lyric, s_lyric])
        if debug: print('PREFIX: ' ,prefix)
        if len(prefix) >= 4: 
            if debug: print('STARTS THE SAME')
            match_end = len(prefix)
            found = True

    if found:
        c_line['start'] = s_line['start']

        # If not exact match, update the start time for updated synced line
        if s_lyric[match_end:].strip():
            pattern = r"((\d{2}):(\d{2}):(\d{2}))"
            time = re.findall(pattern, s_line['delta'])[0]

            delta_as_int = int(time[1]) * 60 + int(time[2]) + int(time[3]) * 0.01
            ratio = float(match_end / float(len(s_lyric)))


            duration = delta_as_int * ratio
            duration = datetime.timedelta(seconds=duration)
            minute = int(duration.total_seconds()//60)
            sec = int(duration.total_seconds()%60)
            tenths = int(round((duration.total_seconds() % 1) * 100))
            c_line['delta'] = f"{minute:02}:{sec:02}:{tenths:02}"

            new_end_time = str_to_time(s_line['start'], '.') + duration
            new_end_time = f"{new_end_time.minute:02}:{new_end_time.second:02}:{new_end_time.microsecond//10000:02}"
            c_line['end'] = new_end_time
            
            duration = delta_as_int - delta_as_int * ratio
            duration = datetime.timedelta(seconds=duration)
            minute = int(duration.total_seconds()//60)
            sec = int(duration.total_seconds()%60)
            tenths = int(round((duration.total_seconds() % 1) * 100))
            s_line['delta'] = f"{minute:02}:{sec:02}:{tenths:02}"

        else:
            c_line['end'] = s_line['end']
            c_line['delta'] = getDelta(c_line, ':')
            
        c_line['lyric'] = c_line_og['lyric']

        s_line['lyric'] = s_lyric[match_end:].strip()
        s_line['start'] = c_line['end']

        if debug: print(c_line)
        if debug: print(s_line)

    return {'short_line': c_line, 'long_line': s_line, 'found': found}


In [14]:
def look_ahead_synced(s_line_og, c_line_og, debug):
    c_line = c_line_og.copy()
    c_lyric = normalize_lyrics(c_line['lyric'].lower()).strip()

    s_line = s_line_og.copy()
    s_line['member'] = 'UNKNOWN'
    s_line['section'] = 'UNKNOWN'
    s_line['line'] = 'UNKNOWN'
    s_lyric = normalize_lyrics(s_line['lyric'].lower()).strip()


    if debug: print('s_line_before', s_line)
    if debug: print('c_line_before',c_line)
    
    found = False
    match_end = 0

    max_l_dist = 5
    if len(c_lyric) < 5 or len(s_lyric) < 5:
        max_l_dist = 1
    elif len(c_lyric) < 10 or len(s_lyric) < 10:
        max_l_dist = int(min(len(c_lyric), len(s_lyric)) / 3)
    
    if debug: print('max dist', max_l_dist, int(min(len(c_lyric), len(s_lyric))))
    find_match = find_near_matches(s_lyric, c_lyric, max_l_dist=max_l_dist)
    if debug: print('find_match: ', find_match)

    if find_match:
        found = find_match[0].start <= 3 and find_match[0].matched
        match_end = find_match[0].end
    else:
        prefix = os.path.commonprefix([c_lyric, s_lyric])
        if debug: print('PREFIX: ' ,prefix)
        if len(prefix) >= 4: 
            if debug: print('STARTS THE SAME')
            match_end = len(prefix)
            found = True

    if found:
        pattern = r"((\d{2}):(\d{2}):(\d{2}))"
        time = re.findall(pattern, s_line['delta'])[0]

        delta_as_int = int(time[1]) * 60 + int(time[2]) + int(time[3]) * 0.01
        ratio = float(match_end) / float(len(c_lyric))

        duration = delta_as_int * ratio
        duration = datetime.timedelta(seconds=duration)
        minute = int(duration.total_seconds()//60)
        sec = int(duration.total_seconds()%60)
        tenths = int(round((duration.total_seconds() % 1) * 100))
        s_line['delta'] = f"{minute:02}:{sec:02}:{tenths:02}"

        new_end_time = str_to_time(s_line['start'], '.') + duration
        new_end_time = f"{new_end_time.minute:02}:{new_end_time.second:02}:{new_end_time.microsecond//10000:02}"
        # s_line['end'] = new_end_time
        s_line['member'] = c_line['member']
        s_line['section'] = c_line['section']
        s_line['line'] = c_line['line']
        
        c_line['lyric'] = c_lyric[match_end:].strip()
        if debug: print('s_line', s_line)
        if debug: print('c_line',c_line)

        
    return {'short_line': s_line, 'long_line': c_line, 'found': found}


In [24]:
def merge_singular(colored_lyrics, synced_lyrics, looking_ahead_max, debug):    
    merged_lines = []

    unmerged_line_color = []
    unmerged_line_synced = []

    c_i = 0
    s_i = 0

    # colored_lyrics_copy = colored_lyrics[78:100].copy()
    # synced_lyrics_copy = synced_lyrics[50:70].copy()

    colored_lyrics_copy = colored_lyrics.copy()
    synced_lyrics_copy = synced_lyrics.copy()

    looking_ahead_index = 0
    looking_ahead = []

    find_children = False
    parent = True

    default_merge_choice_is_color = False

    if len(colored_lyrics) > len(synced_lyrics):
        default_merge_choice_is_color = True
    else:
        default_merge_choice_is_color = False

    if debug: print(default_merge_choice_is_color, len(colored_lyrics_copy), len(synced_lyrics_copy))

    while c_i < len(colored_lyrics_copy) or s_i < len(synced_lyrics_copy):
        
        if debug: print('c_i',c_i, '  s_i', s_i, '  looking_ahead_index', looking_ahead_index)

        if c_i >= len(colored_lyrics_copy):
            if debug: print('MERGE REMAINING SYNCED')
            for s in synced_lyrics_copy[s_i:]:
                unmerged_line_synced.append(s)
                if debug: print('ADDING UMERGED SYNCED --->', s)
            break

        if s_i >= len(synced_lyrics_copy):
            if debug: print('MERGE REMAINING COLOR')
            for c in colored_lyrics_copy[c_i:]:
                unmerged_line_synced.append(c)
                if debug: print('ADDING UMERGED COLOR --->', c)
            break
        
        if c_i + looking_ahead_max >= len(colored_lyrics_copy):
            if debug: print("OVER")
            looking_ahead_max = max(0, len(colored_lyrics_copy) - c_i - 1)
            if debug: print(looking_ahead_max, c_i, s_i)
            # break
        elif s_i + looking_ahead_max >= len(synced_lyrics_copy):
            if debug: print("OVER")
            looking_ahead_max = max(0, len(synced_lyrics_copy) - s_i - 1)
            if debug: print(looking_ahead_max, c_i, s_i)
            # break
        
        c_line = colored_lyrics_copy[c_i + looking_ahead_index]
        s_line = synced_lyrics_copy[s_i]

        c_lyric = c_line['lyric']
        c_lyric_norm = normalize_lyrics(c_lyric)
        s_lyric = s_line['lyric']
        s_lyric_norm = normalize_lyrics(s_lyric)
        if debug: print(c_lyric_norm, s_lyric_norm)

        if c_lyric_norm == s_lyric_norm:
            if debug: print(c_lyric, s_lyric)
            new_line = {'start': s_line['start'], 'end': s_line['end'], 'lyric': c_lyric, 'member': c_line['member'], 'delta': s_line['delta'], 'section': c_line['section'], 'line': c_line['line']}
            merged_lines.append(new_line)
            
            if debug: print("EXACT MATCH: ", new_line)
            if debug: print()
            c_i += (looking_ahead_index + 1)
            s_i += 1
            looking_ahead_index = 0
        else:
            color_is_short = True
            if debug: print("LENGETH: ",len(c_lyric_norm),len(s_lyric_norm))
            if len(c_lyric_norm) <= len(s_lyric_norm):
                if debug: print('color shorter')
                looking_ahead = look_ahead_color(c_line, s_line, debug)
            else:
                color_is_short = False
                if debug: print('synced shorter')
                looking_ahead = look_ahead_synced(s_line, c_line, debug)

            if looking_ahead_index:
                parent = color_is_short

            if debug: print('LOOK AROUND --->', looking_ahead)
            if looking_ahead['found']:
                merged_lines.append(looking_ahead['short_line'])
                if debug: print('MERGE --->', looking_ahead['short_line'])

                if (looking_ahead_index <= looking_ahead_max) :
                    if debug: print("inc colored inside")
                    for l in range(looking_ahead_index):
                        unmerged_line_color.append(colored_lyrics_copy[c_i+l])
                        if debug: print('ADDING UMERGED COLOR: ', l, ' --->', colored_lyrics_copy[c_i+l])
                            
                if color_is_short:
                    if not looking_ahead['long_line']['lyric']:
                        s_i += 1
                        c_i += (looking_ahead_index + 1)
                        if debug: print("EXACT MATCH")
                        find_children = False
                                
                    else:
                        old = synced_lyrics_copy[s_i]
                        synced_lyrics_copy[s_i] = looking_ahead['long_line']
                        if debug: print("PARTIAL MATCH ---> replacing in synced ", old, synced_lyrics_copy[s_i], ' at ', s_i)

                        c_i += (1 + looking_ahead_index)
                        find_children = True
                else:
                    if not looking_ahead['long_line']['lyric']:
                        c_i += (1 + looking_ahead_index)
                        s_i += 1
                        if debug: print("EXACT MATCH")
                        find_children = False
                    else:
                        old = colored_lyrics_copy[c_i + looking_ahead_index]
                        colored_lyrics_copy[c_i + looking_ahead_index] = looking_ahead['long_line']
                        if debug: print("PARTIAL MATCH ---> replacing in colored ", old, colored_lyrics_copy[c_i + looking_ahead_index], ' at ', c_i + looking_ahead_index)
                        s_i += 1
                        c_i += looking_ahead_index
                        find_children = True

                looking_ahead_index = 0
            else:
                if debug: print('not found', looking_ahead_index)
                looking_ahead_index += 1

        if looking_ahead_index > looking_ahead_max:
            if find_children:
                if parent:
                    if debug: print("inc colored")
                    c_i += 1
                    unmerged_line_color.append(colored_lyrics_copy[c_i])
                    if debug: print('ADDING UMERGED COLOR --->', colored_lyrics_copy[c_i])
                else:
                    if debug: print("inc short")
                    unmerged_line_synced.append(synced_lyrics_copy[s_i])
                    if debug: print('ADDING UMERGED SYNCED --->', synced_lyrics_copy[s_i])
                    s_i += 1
            else:
                if debug: print('ADDING UMERGED COLOR --->', colored_lyrics_copy[c_i])
                if debug: print('ADDING UMERGED SYNCED --->', synced_lyrics_copy[s_i])

                unmerged_line_color.append(colored_lyrics_copy[c_i])
                unmerged_line_synced.append(synced_lyrics_copy[s_i])
                c_i += 1
                s_i += 1
            find_children = False
            if debug: print('rest looking_ahead_index')
            
            looking_ahead_index = 0
        
        if debug: print()

    key_order = ['start', 'end', 'delta', 'section', 'line', 'member', 'lyric']

    merged_lines = [
        {k: entry[k] for k in key_order if k in entry} 
        for entry in merged_lines
    ]
    unmerged_line_color = [
        {k: entry[k] for k in key_order if k in entry} 
        for entry in unmerged_line_color
    ]
    unmerged_line_synced = [
        {k: entry[k] for k in key_order if k in entry} 
        for entry in unmerged_line_synced
    ]

        
    return {"merged_lines": merged_lines, "unmerged_line_color": unmerged_line_color, "unmerged_line_synced": unmerged_line_synced}

merge_singular(colored_lyrics, synced_lyrics, 1, True)

True 80 29
c_i 0   s_i 0   looking_ahead_index 0
흩어진조각들이모일때 흩어진조각들이모일때
흩어진 조각들이 모일 때 흩어진 조각들이 모일 때
EXACT MATCH:  {'start': '00:13:08', 'end': '00:16:28', 'lyric': '흩어진 조각들이 모일 때', 'member': ['Jun'], 'delta': '00:03:20', 'section': 0, 'line': 0}


c_i 1   s_i 1   looking_ahead_index 0
비로소난숨을쉬어 비로소난숨을쉬어
 비로소 난 숨을 쉬어 비로소 난 숨을 쉬어
EXACT MATCH:  {'start': '00:16:28', 'end': '00:18:68', 'lyric': ' 비로소 난 숨을 쉬어', 'member': ['Jun'], 'delta': '00:02:40', 'section': 0, 'line': 1}


c_i 2   s_i 2   looking_ahead_index 0
두개의빛마주할때 두개의빛마주할때
 두 개의 빛 마주할 때 두 개의 빛 마주할 때
EXACT MATCH:  {'start': '00:19:58', 'end': '00:22:82', 'lyric': ' 두 개의 빛 마주할 때', 'member': ['Jun'], 'delta': '00:03:24', 'section': 0, 'line': 2}


c_i 3   s_i 3   looking_ahead_index 0
서로의내일을밝혀 서로의내일을밝혀
 서로의 내일을 밝혀 서로의 내일을 밝혀
EXACT MATCH:  {'start': '00:22:82', 'end': '00:25:18', 'lyric': ' 서로의 내일을 밝혀', 'member': ['Jun'], 'delta': '00:02:36', 'section': 0, 'line': 3}


c_i 4   s_i 4   looking_ahead_index 0
해지고어둠이찾아올땐 해지고어둠이찾아올땐오직난너의온도를기억

{'merged_lines': [{'start': '00:13:08',
   'end': '00:16:28',
   'delta': '00:03:20',
   'section': 0,
   'line': 0,
   'member': ['Jun'],
   'lyric': '흩어진 조각들이 모일 때'},
  {'start': '00:16:28',
   'end': '00:18:68',
   'delta': '00:02:40',
   'section': 0,
   'line': 1,
   'member': ['Jun'],
   'lyric': ' 비로소 난 숨을 쉬어'},
  {'start': '00:19:58',
   'end': '00:22:82',
   'delta': '00:03:24',
   'section': 0,
   'line': 2,
   'member': ['Jun'],
   'lyric': ' 두 개의 빛 마주할 때'},
  {'start': '00:22:82',
   'end': '00:25:18',
   'delta': '00:02:36',
   'section': 0,
   'line': 3,
   'member': ['Jun'],
   'lyric': ' 서로의 내일을 밝혀'},
  {'start': '00:26:80',
   'end': '00:29:59',
   'delta': '00:02:79',
   'section': 1,
   'line': 0,
   'member': ['Dino'],
   'lyric': '해지고 어둠이 찾아올 땐'},
  {'start': '00:29:59',
   'end': '00:32:66',
   'delta': '00:03:07',
   'section': 1,
   'line': 1,
   'member': ['Dino'],
   'lyric': ' 오직 난 너의 온도를 기억해'},
  {'start': '00:32:66',
   'end': '00:36:36',
   'delta': '00:03

In [35]:
def main():
    with open('../json/urls.json', 'r') as file:
        data = json.load(file)

    print(data)

    overall_data = []
    debug = False

    for d in data:
        song_name = d['name']
        artist = 'seventeen'
        album = d['album']

        print(song_name)
        raw_color_coded = get_raw_color_coded_html(song_name)
        colored_lyrics = get_colored_lyrics(raw_color_coded)
        export_testing_html(str(raw_color_coded['main_lyrics_body']))
        meta_data = get_song_info(song_name, artist, album)

        if meta_data == 'song not found':
            print('song not found')
            continue
        
        meta_data['color_key'] = raw_color_coded['color_key']
            
        meta_data['unit'] = raw_color_coded['unit']
        synced_lyrics = meta_data['syncedLyrics']

        looking_ahead_max = 1
        merged = merge_singular(colored_lyrics, synced_lyrics, looking_ahead_max, debug)
        meta_data['syncedLyrics'] = merged['merged_lines']
        meta_data['unmerged_line_color'] = merged['unmerged_line_color']
        meta_data['unmerged_line_synced'] = merged['unmerged_line_synced']
        
        song_name = song_name.replace(' ', '_')
        with open('../json/songs/'+ meta_data['name'].lower() +'.json', 'w') as file:
                json.dump(meta_data, file, indent=4)

        print()
    
    print('done')

    #     overall_data.append(meta_data)

    # with open('../json/meta_data.json', 'w') as file:
    #         json.dump(overall_data, file, indent=4)
main()

[{'name': 'Rock with you', 'url': 'https://colorcodedlyrics.com/2021/10/21/seventeen-rock-with-you/', 'album': "SEVENTEEN 9th Mini Album 'Attacca' - SEVENTEEN"}, {'name': 'VERY NICE', 'url': 'https://colorcodedlyrics.com/2016/07/03/seventeen-very-nice-aju-nice/', 'album': "SEVENTEEN 1st Mini Album '17 CARAT' - SEVENTEEN"}, {'name': 'Shining Diamond', 'url': 'https://colorcodedlyrics.com/2015/06/01/seventeen-sebeuntin-shining-diamond/', 'album': "SEVENTEEN 1st Mini Album '17 CARAT' - SEVENTEEN"}, {'name': 'Super', 'url': 'https://colorcodedlyrics.com/2023/04/24/seventeen-super-sonogong/', 'album': "SEVENTEEN 10th Mini Album 'FML' - EP - SEVENTEEN"}, {'name': 'HOT', 'url': 'https://colorcodedlyrics.com/2022/05/27/seventeen-hot/', 'album': "SEVENTEEN 4th Album 'Face the Sun' - SEVENTEEN"}, {'name': 'CHEERS', 'url': 'https://colorcodedlyrics.com/2022/07/18/seventeen-leaders-cheers/', 'album': "SEVENTEEN 4th Album Repackage 'SECTOR 17' - SEVENTEEN"}, {'name': 'To you', 'url': 'https://color

In [31]:
def singular():        
        with open('../json/urls.json', 'r') as file:
                data = json.load(file)

        overall_data = []
        debug = False

        song_name = input()
        artist = 'seventeen'

        song = next((s for s in data if s["name"] == song_name), None)
        print(song)
        album = song['album']

        raw_color_coded = get_raw_color_coded_html(song_name)
        colored_lyrics = get_colored_lyrics(raw_color_coded)
        export_testing_html(str(raw_color_coded['main_lyrics_body']))
        meta_data = get_song_info(song_name, artist, album)

        meta_data['color_key'] = raw_color_coded['color_key']
        
        meta_data['unit'] = raw_color_coded['unit']
        synced_lyrics = meta_data['syncedLyrics']

        looking_ahead_max = 1
        merged = merge_singular(colored_lyrics, synced_lyrics, looking_ahead_max, debug)
        meta_data['syncedLyrics'] = merged['merged_lines']
        meta_data['unmerged_line_color'] = merged['unmerged_line_color']
        meta_data['unmerged_line_synced'] = merged['unmerged_line_synced']

        song_name = song_name.replace(' ', '_')
        with open('../json/songs/'+ meta_data['name'].lower() +'.json', 'w') as file:
                json.dump(meta_data, file, indent=4)
singular()

{'name': 'LOVE, MONEY, FAME', 'url': 'https://colorcodedlyrics.com/2024/10/14/seventeen-love-money-fame-feat-dj-khaled/', 'album': "SEVENTEEN 12th Mini Album 'SPILL THE FEELS' - SEVENTEEN"}
url found: https://colorcodedlyrics.com/2024/10/14/seventeen-love-money-fame-feat-dj-khaled/
requesting  https://lrclib.net/api/get?artist_name=seventeen&track_name=LOVE,_MONEY,_FAME&albumName=SEVENTEEN_12th_Mini_Album_'SPILL_THE_FEELS'_-_SEVENTEEN
../official_times/LOVE, MONEY, FAME.srt found


In [ ]:
# print(raw_color_coded)

In [36]:
directory_path = Path('../json/songs')
files = [p for p in directory_path.iterdir() if p.is_file()]

meta_data = []
for file in files:
    print(file)

    with open(file, 'r') as f:
        data = json.load(f)

    meta_data.append(data)

# with open('../json/songs/lalali.json', 'r') as f:
#     meta_data = json.load(f)

with open('../json/meta_data.json', 'w') as file:
        json.dump(meta_data, file, indent=4)

../json/songs/lalali.json
../json/songs/lovemoneyfame.json
../json/songs/badinfluence.json
../json/songs/leftright.json
../json/songs/rockwithyou.json
../json/songs/shiningdiamond.json
../json/songs/cheerstoyouth.json
../json/songs/thanks.json
../json/songs/hot.json
../json/songs/toyou.json
../json/songs/spell.json
../json/songs/godofmusic.json
../json/songs/darling.json
../json/songs/fckmylife.json
../json/songs/cheers.json
../json/songs/_world.json
../json/songs/highlight.json
../json/songs/verynice.json
../json/songs/super.json
